<a href="https://colab.research.google.com/github/sire-ambrose/FallingKnife/blob/main/falling_knives_ml.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install afrimarket
!pip3 install backtesting
#!pip3 install TA-Lib
!pip3 install ta
!pip3 install pyfolio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 22.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.3/38.3 MB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 16.6 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
  Attempting uninstall: lxml
    Found existing installation: lxml 4.9.4
    Uninstalling lxml-4.9.4:
      Successfully uninstalled lxml-4.9.4
  Attempting uninstall: pandas
    Found existing installation: pandas 2.0.3
    Uninstalling pandas-2.0.3:
      Successfully uninstalled pandas-2.0.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 15.0.2 which is incompatible.
google-colab 1.0.0 requir

In [ ]:
import afrimarket as afm
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
from sklearn.preprocessing import StandardScaler
from statsmodels.tsa.stattools import adfuller
from sklearn.utils import check_random_state
import matplotlib.pyplot as plt
import ta
import plotly.graph_objects as go
from imblearn.over_sampling import SMOTE

# For ignoring warnings
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Function to create the DC indicators provided by Chen and Tsang (2021)
def directional_change_events(data, theta=0.2):

    # Copy the dataframe
    data = data.copy()

    # Create the necessary columns and variables
    data["Event"] = 0.0

    # Set the initial event variable value
    event = "upward" # initial event

    # Set the initial value for low and high prices
    ph = data['Close'].iloc[0] # highest price
    pl = data['Close'].iloc[0] # lowest price

    # Create loop to run through each date
    for t in range(0, len(data.index)):
        # Check if we're on a downward trend
        if event == "downward":
            # Check if the close price is higher than the low price by the theta threshold
            if data["Close"].iloc[t] >= pl * (1 + theta):
                # Set the event variable to upward
                event = "upward"
                # Set the high price as the current close price
                ph = data["Close"].iloc[t]
            # If the close price is lower than the low price by the theta threshold
            else:
                # Check if the close price is less than the low price
                if data["Close"].iloc[t] < pl:
                    # Set the low price as the current close price
                    pl = data["Close"].iloc[t]
                    # Set the Event to upward for the current period
                    data["Event"].iloc[t] = 1
        # Check if we're on a upward trend
        elif event == "upward":
            # Check if the close price is less than the high price by the theta threshold
            if data["Close"].iloc[t] <= ph * (1 - theta):
                # Set the event variable to downward
                event = "downward"
                # Set the low price as the current close price
                pl = data["Close"].iloc[t]
            # If the close price is higher than the high price by the theta threshold
            else:
                # Check if the close price is higher than the high price
                if data["Close"].iloc[t] > ph:
                    # Set the high price as the current close price
                    ph = data["Close"].iloc[t]
                    # Set the Event to downward for the current period
                    data["Event"].iloc[t] = -1

    # Set the peak and trough prices and forward-fill the column
    data['peak_trough_prices'] = np.where(data['Event']!=0, data['Close'],0)
    data['peak_trough_prices'].replace(to_replace=0, method='ffill', inplace=True)

    # Count the number of periods between a peak and a trough
    data['count'] = 0
    for i in range(1,len(data.index)):
        if data['Event'].iloc[(i-1)]!=0:
            data['count'].iloc[i] = 1+data['count'].iloc[(i-1)]
        else:
            data['count'].iloc[i] = 1

    # Compute the TMV indicator
    data['TMV'] = np.where(data['Event']!=0, abs(data['peak_trough_prices']-data['peak_trough_prices'].shift())/\
                          (data['peak_trough_prices'].shift()*theta),0)

    # Compute the time-completion-for-a-trend indicator
    data['T'] = np.where(data['Event']!=0, data['count'],0)

    # Compute the time-adjusted-return indicator and forward-fill it
    data['R'] = np.where(data['Event']!=0, np.log(data['TMV']/data['T']*theta),0)
    data['R'] = data['R'].replace(to_replace=0, method='ffill')

    # Drop NaN or inifinite values
    data.replace([np.inf, -np.inf], np.nan, inplace=True)

    return data


In [ ]:
market= afm.Exchange(market='ngx')
asi= market.get_index_price()
asi.index= asi.Date
asi.drop(['Date'], axis=1, inplace=True)
asi.rename({'Price':'close'}, axis=1, inplace=True)
asi

,close
Date,
2014-06-23,41577.47
2014-06-24,41740.52
2014-06-25,41987.55
2014-06-26,41729.59
2014-06-27,42187.62
...,...
2024-06-11,99630.51
2024-06-13,99832.25
2024-06-14,99925.29


In [ ]:
df= directional_change_events(asi.rename({'close':'Close'}, axis=1).copy(), theta=0.2)
target= df['Event']
df.head()

,Close,Event,peak_trough_prices,count,TMV,T,R
Date,,,,,,,
2014-06-23,41577.47,0.0,0.00,0,0.000000,0,0.000000
2014-06-24,41740.52,-1.0,41740.52,1,NaN,1,NaN
2014-06-25,41987.55,-1.0,41987.55,2,0.029591,2,-5.822865
2014-06-26,41729.59,0.0,41987.55,3,0.000000,0,-5.822865
2014-06-27,42187.62,-1.0,42187.62,1,0.023825,1,-5.346461


In [ ]:
def update_event_column(df):
  N= 2
  for i in range(1, len(df)):
    for j in range(N+1, 0, -1):
      if i>j:
        if all(df['Event'].iloc[i-j:i] == -1) and df['Event'].iloc[i] == -1 :
            df.iloc[i-j:i, df.columns.get_loc('Event')] = 0
            break
        elif all(df['Event'].iloc[i-j:i] == 1) and df['Event'].iloc[i] == 1:
            df.iloc[i-j:i, df.columns.get_loc('Event')] = 0
            break

update_event_column(df)

df.head(10)

,Close,Event,peak_trough_prices,count,TMV,T,R
Date,,,,,,,
2014-06-23,41577.47,0.0,0.00,0,0.000000,0,0.000000
2014-06-24,41740.52,0.0,41740.52,1,NaN,1,NaN
2014-06-25,41987.55,-1.0,41987.55,2,0.029591,2,-5.822865
2014-06-26,41729.59,0.0,41987.55,3,0.000000,0,-5.822865
2014-06-27,42187.62,0.0,42187.62,1,0.023825,1,-5.346461
2014-06-30,42482.48,0.0,42482.48,2,0.034946,2,-5.656529
2014-07-01,42611.30,-1.0,42611.30,3,0.015162,3,-6.897043
2014-07-02,42609.75,0.0,42611.30,4,0.000000,0,-6.897043
2014-07-03,42686.58,0.0,42686.58,1,0.008833,1,-6.338660


In [ ]:
fig = px.line(asi, x=asi.index, y='close', title='ASI')
fig.update_xaxes(title_text='Date')
fig.update_yaxes(title_text='Close')

scatter = go.Scatter(x=df.index, y=(df['peak_trough_prices']*df['Event']).abs().replace(0, np.nan), mode='markers', name='Peak Trough Prices')
fig.add_trace(scatter)

fig.show()

In [ ]:
# Function to check stationarity
def check_stationarity(df, column):
    result = adfuller(df[column])
    print(f'ADF Statistic for {column}: {result[0]}')
    print(f'p-value for {column}: {result[1]}')
    if result[1] > 0.05:
        print(f'{column} is not stationary. Applying differencing...')
        df[column] = df[column].diff().dropna()
    else:
        print(f'{column} is stationary.')

# Function to normalize data
def normalize_data(df, columns):
    scaler = StandardScaler()
    df[columns] = scaler.fit_transform(df[columns])
    return df, scaler

In [ ]:
# Normalize data
asi, scaler = normalize_data(asi, ['close'])

In [ ]:
def trends(df, indicators, timestamp=True):

    ### TRENDS

    # DPO
    dpo= ta.trend.DPOIndicator(close=df.close, window=20, fillna=False)
    indicators['dpo']= dpo.dpo()

    # EMA
    ema= ta.trend.EMAIndicator(close=df['close'], window=14, fillna = False)
    indicators['ema_indicator']= ema.ema_indicator()

    # KST
    kst= ta.trend.KSTIndicator(close=df['close'], roc1 = 10, roc2 = 15, roc3= 20, roc4 = 30, window1= 10, window2= 10, window3= 10, window4= 15, nsig= 9, fillna= False)
    indicators['kst']=kst.kst()
    indicators['kst_diff']=kst.kst_diff()
    indicators['kst_sig']=kst.kst_sig()

    # MACD
    macd= ta.trend.MACD(close=df.close, window_slow= 26, window_fast = 12, window_sign= 9, fillna= False)
    indicators['macd']=macd.macd()
    indicators['macd_diff']=macd.macd_diff()
    indicators['macd_signal']=macd.macd_signal()

    # SMA
    sma= ta.trend.SMAIndicator(close=df['close'], window=30, fillna = False)
    indicators['sma_indicator']= df['close']-sma.sma_indicator()

    # STC
    indicators['stc']=ta.trend.STCIndicator(close=df.close, window_slow= 50, window_fast=23, cycle= 10, smooth1= 3, smooth2 = 3, fillna = False).stc()

    # TRIX
    indicators['trix']=ta.trend.TRIXIndicator(close=df.close, window=15, fillna = False).trix()

    return indicators

In [ ]:
def momentum(df, indicators, timestamp=True):
    ### Momentum

    ### RSI
    indicators['RSI']= ta.momentum.RSIIndicator(close=df['close']).rsi()


    ### Kama Indicator
    indicators['kama_indicator']= ta.momentum.KAMAIndicator(close=df['close'], window= 10, pow1= 2, pow2= 30, fillna = False).kama()-df['close']

    ### Percentage Price Oscillator
    ppo= ta.momentum.PercentagePriceOscillator(close= df['close'], window_slow= 26, window_fast= 12, window_sign= 9, fillna = False)
    indicators['ppo']= ppo.ppo()
    indicators['ppo_signal']=ppo.ppo_signal()

    ### Rate Of Change
    roc= ta.momentum.ROCIndicator(close= df['close'], window= 12, fillna= False)
    indicators['roc']= roc.roc()

    ### Stochastic RSI
    stochrsi=ta.momentum.StochRSIIndicator(close= df['close'], window = 14, smooth1 = 3, smooth2 = 3, fillna= False)
    indicators['stochrsi']=stochrsi.stochrsi()
    indicators['stochrsi_d']= stochrsi.stochrsi_d()
    indicators['stochrsi_k']= stochrsi.stochrsi_k()

    ### True Strength Index
    indicators['true_strength_index']=ta.momentum.TSIIndicator(close=df['close'], window_slow= 25, window_fast = 13, fillna = False).tsi()

    return indicators

In [ ]:
def volatility(df, indicators, timestamp=True):
    ### Volatility

    '''
    # ATR
    indicators['atr']=ta.volatility.average_true_range(high=df.high, low=df.low, close=df.close, window=14, fillna=False)
    '''

    # Bollinger Band
    bollinger=ta.volatility.BollingerBands(close=df['close'], window= 20, window_dev= 2, fillna = False)

    indicators['bollinger_upperband_price_diff']= df['close']-bollinger.bollinger_hband() # if positive, price is above upper band
    indicators['bollinger_hband_indicator']= bollinger.bollinger_hband_indicator()
    indicators['bollinger_lowerband_price_diff']= bollinger.bollinger_lband()- df['close']
    indicators['bollinger_lband_indicator']= bollinger.bollinger_lband_indicator()
    indicators['bollinger_mavg_price_diff']= bollinger.bollinger_mavg()- df['close']
    indicators['bollinger_pband']= bollinger.bollinger_pband()
    indicators['bollinger_wband']= bollinger.bollinger_wband()

    # Ulcer Index
    indicators['ulcer_index']=ta.volatility.ulcer_index(close=df.close, window=14, fillna=False)
    if timestamp==True:
        indicators['timestamp']=df['timestamp']
    return indicators

In [ ]:
indicators= pd.DataFrame()

indicators=trends(asi, indicators)
indicators=momentum(asi, indicators)
indicators=volatility(asi, indicators, timestamp=False)
indicators

,dpo,ema_indicator,kst,kst_diff,kst_sig,macd,macd_diff,macd_signal,sma_indicator,stc,...,stochrsi_k,true_strength_index,bollinger_upperband_price_diff,bollinger_hband_indicator,bollinger_lowerband_price_diff,bollinger_lband_indicator,bollinger_mavg_price_diff,bollinger_pband,bollinger_wband,ulcer_index
Date,,,,,,,,,,,,,,,,,,,,,
2014-06-23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.0,NaN,0.0,NaN,NaN,NaN,NaN
2014-06-24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.0,NaN,0.0,NaN,NaN,NaN,NaN
2014-06-25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.0,NaN,0.0,NaN,NaN,NaN,NaN
2014-06-26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.0,NaN,0.0,NaN,NaN,NaN,NaN
2014-06-27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.0,NaN,0.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-06-11,-0.043371,3.477846,-0.605411,19.497443,-20.102854,0.005668,0.013063,-0.007395,0.064794,99.951172,...,0.942285,3.934499,-0.025059,0.0,-0.146012,0.0,-0.060477,0.853518,4.957516,0.912881
2024-06-13,-0.019760,3.483876,3.746892,19.282731,-15.535839,0.008580,0.012780,-0.004200,0.072824,99.975586,...,0.962331,6.333094,-0.019995,0.0,-0.150800,0.0,-0.065403,0.882929,4.939622,0.560788
2024-06-14,-0.001783,3.489830,7.818844,18.780784,-10.961941,0.011199,0.012319,-0.001120,0.074963,99.987793,...,0.962331,8.622776,-0.018933,0.0,-0.145501,0.0,-0.063284,0.884859,4.745236,0.262104


In [ ]:
from sklearn.model_selection import train_test_split, TimeSeriesSplit
from sklearn.preprocessing import LabelEncoder

In [ ]:
asi

,close
Date,
2014-06-23,0.099741
2014-06-24,0.109323
2014-06-25,0.123839
2014-06-26,0.108681
2014-06-27,0.135597
...,...
2024-06-11,3.511213
2024-06-13,3.523068
2024-06-14,3.528536


In [ ]:
asi.shift(-1)

,close
Date,
2014-06-23,0.109323
2014-06-24,0.123839
2014-06-25,0.108681
2014-06-26,0.135597
2014-06-27,0.152924
...,...
2024-06-11,3.523068
2024-06-13,3.528536
2024-06-14,3.523580


In [ ]:
together= pd.concat([indicators, target.shift(-1)], axis=1).dropna()

together['Event'].replace(-1, 2, inplace=True)

test_size= int(0.3*together.shape[0])

tss= TimeSeriesSplit(n_splits=2, test_size=test_size)

for i, (train_index, test_index) in enumerate(tss.split(together)):
  train, test = together.iloc[train_index], together.iloc[test_index]

cols= ['Event', 'bollinger_hband_indicator', 'bollinger_lband_indicator']
x_train, x_test, y_train, y_test= train.drop(cols, axis=1), test.drop(cols, axis=1), train[['Event']], test[['Event']]

In [ ]:
X_train, Y_train= SMOTE(random_state=42).fit_resample(x_train, y_train)

In [ ]:
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier

In [ ]:
model= LGBMClassifier(random_state=42)

In [ ]:
model.fit(X_train, Y_train)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.014339 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 6630
[LightGBM] [Info] Number of data points in the train set: 4785, number of used features: 26
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf


LGBMClassifier(random_state=42)

In [ ]:
y_pred= model.predict(x_test)

In [ ]:
y_test.value_counts()

Event
0.0      675
2.0       36
Name: count, dtype: int64

In [ ]:
pd.DataFrame(y_pred).value_counts()

0  
0.0    704
2.0      7
Name: count, dtype: int64

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         0.0       0.95      0.99      0.97       675
         2.0       0.14      0.03      0.05        36

    accuracy                           0.94       711
   macro avg       0.55      0.51      0.51       711
weighted avg       0.91      0.94      0.92       711

